## Guardrails for Amazon Bedrock - Examples using the Python SDK

----------------------------
Guardrails for Amazon Bedrock evaluates user inputs and FM responses based on use case specific policies, and provides an additional layer of safeguards regardless of the underlying FM. Guardrails can be applied across all large language models (LLMs) on Amazon Bedrock, including fine-tuned models. Customers can create multiple guardrails, each configured with a different combination of controls, and use these guardrails across different applications and use cases. 


In [1]:
#Start by installing the dependencies to ensure we have a recent version
!pip install --upgrade --force-reinstall boto3
import boto3
print(boto3.__version__)

Defaulting to user installation because normal site-packages is not writeable
  Using cached boto3-1.35.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.35.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached boto3-1.35.0-py3-none-any.whl (139 kB)
Using cached botocore-1.35.0-py3-none-any.whl (12.5 MB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached urllib3-1.26.19-py2.py3-none-any.whl (143 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: urllib3
    Found ex

In [2]:
REGION_NAME = "us-east-1"
client = boto3.client("bedrock", region_name=REGION_NAME)

#### Creating a Guardrail

Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html) 

In [4]:
create_response = client.create_guardrail(
    name='masking-guardrail',
    description='Prevents the our model from providing fiduciary advice.',
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'ANONYMIZE'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ]
        ,
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    tags=[
        {'key': 'purpose', 'value': 'masking-guardrail1'},
        {'key': 'environment', 'value': 'production'}
    ]
)

print(create_response)

{'ResponseMetadata': {'RequestId': '8defe1a0-dfe2-4dee-873f-47b6ddaad1e4', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 19 Aug 2024 15:48:24 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': '8defe1a0-dfe2-4dee-873f-47b6ddaad1e4'}, 'RetryAttempts': 0}, 'guardrailId': 'x7gl09u5osna', 'guardrailArn': 'arn:aws:bedrock:us-east-1:771855297262:guardrail/x7gl09u5osna', 'version': 'DRAFT', 'createdAt': datetime.datetime(2024, 8, 19, 15, 48, 23, 854837, tzinfo=tzutc())}


### Testing our Guardrail

In [5]:
#import the run-time client
import json
bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION_NAME)

# Build our request to Bedrock, we will test our second version

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Give me example of SSN Number in usa?"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = create_response['guardrailId'], 
    guardrailVersion ="DRAFT", 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

{
  "id": "msg_bdrk_011WJCeoNhKHshrGPGwPWtfE",
  "model": "claude-3-haiku-20240307",
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "Here is an example of a Social Security Number (SSN) in the USA:\n\n{US_SOCIAL_SECURITY_NUMBER}\n\nThe key features of a Social Security number in the USA are:\n\n1. It is a 9-digit number, divided into 3 parts:\n   - First 3 digits: This is the area number, which indicates the state or region where the number was issued.\n   - Middle 2 digits: This is the group number, which is used to further organize the numbers.\n   - Last 4 digits: This is the serial number, which is a unique identifier.\n\n2. The digits cannot start with 0 or 9 in the first 3 positions.\n\n3. The number is issued by the Social Security Administration (SSA) to U.S. citizens, permanent residents, and temporary (working) residents.\n\n4. It is used for a variety of purposes, such as filing tax returns, applying for government benefi